# Baseline - Most popular
Use max count of user ratings in test set for precision and recall calculation

In [38]:
import importlib
from collections import defaultdict

import pandas as pd
from sklearn.model_selection import train_test_split

import bibrec.server.Utils as Utils
import bibrec.server.evaluation as eval
from bibrec.server.Utils import assign_popular_based_score

importlib.reload(eval)
importlib.reload(Utils)

books, users, ratings = Utils.get_normalized_data()

/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=";", encoding="latin-1")
/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:98: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


Normalized Ratings Count: 383962


In [39]:
users_with_ratings = ratings["user_id"].unique()

# normal splitting since user specific splitting doesn't make that much of a difference and takes a long time
train, test = train_test_split(ratings, test_size=0.25)

In [40]:
print("Total Ratings Count:", len(train) + len(test))
print("Train Data Ratings Count:", len(train))
print("Test Data Ratings Count:", len(test))

Total Ratings Count: 383962
Train Data Ratings Count: 287971
Test Data Ratings Count: 95991


In [41]:
book_ratings_dict = {
    'user_id': train['user_id'].values,
    'isbn13': train['isbn13'].values,
    'book_rating': train['book_rating'].values
}

book_rating_df = train[['user_id', 'isbn13', 'book_rating']]

In [42]:
train["isbn13"] = train["isbn13"].astype("int")
books["isbn13"] = books["isbn13"].astype("int")
popular_books = assign_popular_based_score(train, books, "user_id", "isbn13", "book_rating")
popular_books = popular_books.sort_values("vote_count", ascending=False)

print(popular_books[:20])

              isbn13  vote_count  avg_rating  weighted_rating
9236   9780316666343         538    8.232342         8.229756
33631  9780971880108         426    4.462441         4.476795
14372  9780385504201         378    8.449735         8.444917
8100   9780312195519         294    8.248299         8.243474
23631  9780590353427         255    8.964706         8.953573
4765   9780060928339         241    7.987552         7.983821
1787    978044667227         226    8.101770         8.096791
7160   9780142001745         223    8.475336         8.466971
20521  9780452282155         211    7.966825         7.962762
24844  9780671027360         208    8.187500         8.181278
17109  9780440237228         207    7.449275         7.450088
9220   9780316601955         203    7.551724         7.551553
28630  9780743418171         193    8.020725         8.015735
9277   9780316769488         188    7.840426         7.837202
26964  9780679764021         186    7.811828         7.808874
12535  9

In [43]:
pop_dict = {
    'item_id': popular_books['isbn13'].values,
    'est_r': popular_books['avg_rating'].values,
}

pop_df = pd.DataFrame.from_dict(pop_dict)

ratings_dict = {
    'user_id': test['user_id'].values,
    'item_id': test['isbn13'].values,
    'rating': test['book_rating'].values
}

ratings_df = pd.DataFrame.from_dict(ratings_dict)

In [44]:
test_uids = ratings_df["user_id"].unique()

print("User Count:", len(test_uids))

User Count: 29872


In [45]:
def def_value():
    return "Not Present"


top_n = defaultdict(def_value)
for uid in test_uids:
    top_n[uid] = pop_df

In [46]:
import time

importlib.reload(eval)
importlib.reload(Utils)
start_time = time.time()
# use_max_val specifies whether the calculation should use the max count of user ratings as denominator if the recommendation count is higher
avg_precision = eval.get_avg_precision(ratings_df, top_n, k=50, use_max_val=True)

print("Average Precision:", avg_precision)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Precision: 0.018765721994760672
--- Calculation time: 57.0659499168396 seconds ---


In [47]:
importlib.reload(eval)
importlib.reload(Utils)
# use_max_val specifies whether the calculation should use the max count of user ratings as denominator if the relevant item count is higher
avg_recall = eval.get_avg_recall(ratings_df, top_n, k=50, use_max_val=True)

print("Average Recall:", avg_recall)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Recall: 0.022537005694184008
--- Calculation time: 111.04521608352661 seconds ---
